In [2]:
import numpy as np
import netCDF4
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()
%matplotlib inline

In [20]:
#%unload_ext vimception


In [3]:
press = np.array([1000,950,900,850,700,500,400,300,250,200,150,100,70,50,30,20,10])*100
nplevs= len(press)

In [4]:
height_match = np.load('height_match_N96L852000_hossaini_press.npy')            

In [5]:
# get file containing heights, pressures for L85 ancil
filename='/Users/ptg21/data/projects/TRANSCOM/N96_ancil_creation/xkawa.p.avg2000.nc'

ncfile2     = netCDF4.Dataset(filename)

lon2        = ncfile2.variables['longitude'][:]
lat2        = ncfile2.variables['latitude'][:]
pressure    = ncfile2.variables['p'][:]

hybrid_ht2   = ncfile2.variables['hybrid_ht'][:]
times2      = ncfile2.variables['time']
nmonths2, nheights2, ny2, nx2 = np.shape(pressure)

In [6]:
years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]

def read_in_data(year):
    import netCDF4
    filename='/Users/ptg21/data/projects/TRANSCOM/N96_results/regrid_chbr3/all_'+str(year)+'.nc'
    ncfile1    = netCDF4.Dataset(filename)
    lon1        = ncfile1.variables['longitude'][:]
    lat1        = ncfile1.variables['latitude'][:] 
    times1      = ncfile1.variables['t']
    hybrid_ht1  = ncfile1.variables['hybrid_ht'][:]

    field_34062      = ncfile1.variables['field34062'][:,:,:,:] # liang = 62, CH2Br2b
    field_34063      = ncfile1.variables['field34063'][:,:,:,:] # ordonez = 63, CH2Br2c
    field_34065      = ncfile1.variables['field34065'][:,:,:,:] # ziska = 65, CH2Br2a

    nmonths, nlevels, ny1, nx1 = np.shape(field_34062)
    # return liang, ordonez, ziska + dims
    return field_34062, field_34065, field_34063, nmonths, nlevels, ny1, nx1

field_l, field_o, field_z, nmonths, nlevels1, ny1, nx1 = read_in_data(1998)

In [7]:
def interp_lin_val(xnew, x, y, idx):
    # x,y arrays
    # xnew is the new xvalue
    # returns ynew as a linear interpolation
    # returns a value
    dx = xnew - x[idx]
    dy_dx = (y[idx]-y[idx+1])/(x[idx]-x[idx+1]) 
    # calculate a value at thew intermediate pressure
    return y[idx]+dy_dx*dx

In [8]:
liang_res   = np.empty([nmonths, nplevs, ny1, nx1])
ordonez_res = np.empty([nmonths, nplevs, ny1, nx1])
ziska_res   = np.empty([nmonths, nplevs, ny1, nx1])
# tried this using interp1d but was factor of 3 slower!  
years=[1994, 1995, 1996, 1997, 1998, 1999, 2000]
years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009]

nyl=0
nyu = ny1

nxl = 0
nxu = nx1

for year in years:
    print year
    field_l, field_o, field_z, nmonths, nlevels1, ny1, nx1 = read_in_data(year)

    for imonth in range (0, 12):
        print imonth
        for ilat in range(nyl, nyu):
            for ilon in range (nxl, nxu): 
                x_arr = pressure[imonth, :, ilat, ilon]
                for ilev in range(0, nplevs):
                    if height_match[imonth,ilev, ilat, ilon] != -999:
                            liang_res[imonth,  ilev, ilat, ilon] = interp_lin_val(press[ilev], 
                                                                                  x_arr, 
                                                                                  field_l[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])
                            ordonez_res[imonth,ilev, ilat, ilon] =  interp_lin_val(press[ilev], 
                                                                                  x_arr, 
                                                                                  field_o[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])
                            ziska_res[imonth,  ilev, ilat, ilon] = interp_lin_val(press[ilev], 
                                                                                  x_arr,
                                                                                  field_z[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])

                    else:
                        liang_res[imonth,  ilev, ilat, ilon] = -999
                        ordonez_res[imonth,ilev, ilat, ilon] = -999 
                        ziska_res[imonth,  ilev, ilat, ilon] = -999 


    np.save('ordonez_res'+str(year)+'.npy',ordonez_res)
    np.save('liang_res'+str(year)+'.npy',liang_res)
    np.save('ziska_res'+str(year)+'.npy',ziska_res)

1995
0
1
2
3
4
5
6
7
8
9
10
11
1996
0
1
2
3
4
5
6
7
8
9
10
11
1997
0
1
2
3
4
5
6
7
8
9
10
11
1998
0
1
2
3
4
5
6
7
8
9
10
11
1999
0
1
2
3
4
5
6
7
8
9
10
11
2000
0
1
2
3
4
5
6
7
8
9
10
11
2001
0
1
2
3
4
5
6
7
8
9
10
11
2002
0
1
2
3
4
5
6
7
8
9
10
11
2003
0
1
2
3
4
5
6
7
8
9
10
11
2004
0
1
2
3
4
5
6
7
8
9
10
11
2005
0
1
2
3
4
5
6
7
8
9
10
11
2006
0
1
2
3
4
5
6
7
8
9
10
11
2007
0
1
2
3
4
5
6
7
8
9
10
11
2008
0
1
2
3
4
5
6
7
8
9
10
11
2009
0
1
2
3
4
5
6


IndexError: index 6 is out of bounds for axis 0 with size 6

In [9]:
%%bash
ls

N96_L85_CHBr3_tracer1.nc
N96_L85_CHBr3_tracer2.nc
N96_L85_CHBr3_tracer3.nc
create_netcdfs.ipynb
custom.css
find_matching_levels.ipynb
height_match_N96L852000_hossaini_press.npy
interp_1998_N96_results_onto_press_levs_too_slow.ipynb
interp_onto_p_levels.ipynb
liang_res1994.npy
liang_res1995.npy
liang_res1996.npy
liang_res1997.npy
liang_res1998.npy
liang_res1999.npy
liang_res2000.npy
liang_res2001.npy
liang_res2002.npy
liang_res2003.npy
liang_res2004.npy
liang_res2005.npy
liang_res2006.npy
liang_res2007.npy
liang_res2008.npy
mm.UMUKCA.UCam.1.nc.N96L85
mm.UMUKCA.UCam.2.nc.N96L85
mm.UMUKCA.UCam.3.nc.N96L85
new_regrid.tar.gz
ordonez_res1994.npy
ordonez_res1995.npy
ordonez_res1996.npy
ordonez_res1997.npy
ordonez_res1998.npy
ordonez_res1999.npy
ordonez_res2000.npy
ordonez_res2001.npy
ordonez_res2002.npy
ordonez_res2003.npy
ordonez_res2004.npy
ordonez_res2005.npy
ordonez_res2006.npy
ordonez_res2007.npy
ordonez_res2008.npy
ziska_res1994.npy
ziska_res1995.npy
ziska_res1996.npy
ziska_res1997.npy
